In [4]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import os
import requests
import sys 
import shutil
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT DISTINCT ?voiture ?label ?image WHERE {
  VALUES ?type {
    wd:Q274586   # Voitures de sport
    wd:Q185721   # Roadster
    wd:Q34427    # Voiture de collection
    wd:Q3231690  # Pony car (muscle cars légères)
    wd:Q363288   # Kei car (souvent des JDM)
  }
  
  ?voiture wdt:P31 ?type;    # La voiture doit être d'un type défini ci-dessus
           wdt:P18 ?image;   # Image associée
           rdfs:label ?label. # Nom de la voiture

  FILTER(LANG(?label) IN ("fr", "en")).  # Garde les labels en français ou anglais
}
LIMIT 300
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

# Fonction pour télécharger une image
def download_image(url, folder="images"):
    if not os.path.exists(folder):
        os.makedirs(folder)
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_path = os.path.join(folder, os.path.basename(url))
        with open(image_path, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
        return image_path
    return None

# Récupérer les résultats de Wikidata
results = get_results(endpoint_url, query)

# Créer un DataFrame avec les résultats
data = []
for result in results["results"]["bindings"]:
    voiture = result["voiture"]["value"]
    label = result["label"]["value"]
    image_url = result["image"]["value"]
    data.append([label, voiture, image_url])

dataframe = pd.DataFrame(data, columns=["label", "voiture", "image"])

# Télécharger les images
dataframe["image_path"] = dataframe["image"].apply(download_image)

# Afficher le DataFrame avec les chemins d'images
print(dataframe)


                   label                                 voiture  \
0         Shelby Mustang  http://www.wikidata.org/entity/Q282617   
1          Shelby GT 500  http://www.wikidata.org/entity/Q282617   
2          Chevrolet SSR  http://www.wikidata.org/entity/Q282151   
3          Chevrolet SSR  http://www.wikidata.org/entity/Q282151   
4    Volkswagen Jetta II  http://www.wikidata.org/entity/Q282455   
..                   ...                                     ...   
295          Saab Sonett  http://www.wikidata.org/entity/Q420564   
296             Kia Soul  http://www.wikidata.org/entity/Q420780   
297            Saab 9-2X  http://www.wikidata.org/entity/Q420573   
298              Saab 96  http://www.wikidata.org/entity/Q420796   
299            Saab 9-2X  http://www.wikidata.org/entity/Q420573   

                                                 image  \
0    http://commons.wikimedia.org/wiki/Special:File...   
1    http://commons.wikimedia.org/wiki/Special:File...   
2    http

In [16]:
import os
import json
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS

# Dictionnaire pour stocker les métadonnées
metadata = {}

# Fonction pour extraire toutes les métadonnées EXIF
def get_exif(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        exif_info = {}

        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)  # Récupère le nom du tag EXIF
                
                # Vérification du type de la valeur et conversion en format JSON-compatible
                if isinstance(value, bytes):
                    try:
                        value = value.decode("utf-8", errors="ignore")  # Décodage en UTF-8
                    except:
                        value = str(value)  # Conversion brute en string
                
                elif isinstance(value, (int, float, str)):  
                    pass  # Garde la valeur inchangée si elle est déjà compatible
                
                else:
                    value = str(value)  # Convertit en string tout ce qui reste
                
                exif_info[tag_name] = value

        # Ajouter les dimensions et le format de l'image
        width, height = img.size
        orientation = "Paysage" if width > height else "Portrait" if height > width else "Carré"

        exif_info.update({
            "format": img.format,
            "taille": f"{width}x{height}",
            "orientation": orientation
        })

        return exif_info
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction des métadonnées pour {image_path} : {e}")
        return None

# Vérifier si le DataFrame existe et contient des images
if 'dataframe' in locals() and not dataframe.empty:
    for _, row in dataframe.iterrows():
        image_path = row["image_path"]
        if image_path and os.path.exists(image_path):
            metadata[row["label"]] = get_exif(image_path)
else:
    print("⚠️ Aucun fichier image trouvé dans le DataFrame.")

# Sauvegarder toutes les métadonnées en JSON
with open("metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)

print("✅ Extraction des métadonnées terminée !")


✅ Extraction des métadonnées terminée !


In [17]:
import os
import json
import pandas as pd
from PIL import Image
from collections import Counter

# Dossier contenant les images
IMAGE_FOLDER = "images"
OUTPUT_JSON = "image_annotations.json"

# Fonction pour extraire les couleurs dominantes d'une image
def get_dominant_colors(image_path, num_colors=3):
    try:
        img = Image.open(image_path).convert("RGB")  # Convertir en RGB
        img = img.resize((100, 100))  # Réduire la taille pour accélérer l'analyse
        pixels = list(img.getdata())  # Récupérer tous les pixels
        
        counter = Counter(pixels)  # Compter la fréquence des couleurs
        most_common = counter.most_common(num_colors)  # Prendre les couleurs les plus fréquentes
        return [f"rgb{color[0]}" for color in most_common]  # Retourner les couleurs en format RGB
    except Exception as e:
        print(f"Erreur pour {image_path} : {e}")
        return []

# Génération automatique de tags à partir des couleurs dominantes
def generate_tags(colors):
    color_tags = {
        "rgb(255, 0, 0)": "#rouge",
        "rgb(0, 255, 0)": "#vert",
        "rgb(0, 0, 255)": "#bleu",
        "rgb(255, 255, 0)": "#jaune",
        "rgb(0, 0, 0)": "#noir",
        "rgb(255, 255, 255)": "#blanc",
        "rgb(128, 128, 128)": "#gris"
    }
    
    tags = [color_tags.get(c, "#autre") for c in colors if c in color_tags]
    return tags if tags else ["#autre"]

# Demande à l'utilisateur de tagger les images manuellement (optionnel)
def get_manual_tags():
    if os.path.exists("manual_tags.json"):
        with open("manual_tags.json", "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

# Parcourir les images et générer les annotations
def annotate_images():
    annotations = []
    manual_tags = get_manual_tags()  # Charger les tags manuels

    for filename in os.listdir(IMAGE_FOLDER):
        image_path = os.path.join(IMAGE_FOLDER, filename)
        if os.path.isfile(image_path):
            print(f"Analyse de {filename}...")

            # Extraire les couleurs dominantes
            colors = get_dominant_colors(image_path)
            
            # Générer des tags automatiques
            auto_tags = generate_tags(colors)
            
            # Récupérer les tags manuels s'ils existent
            user_tags = manual_tags.get(filename, [])

            # Ajouter les informations dans une liste
            annotations.append({
                "filename": filename,
                "colors": colors,
                "auto_tags": auto_tags,
                "manual_tags": user_tags,
                "all_tags": list(set(auto_tags + user_tags))  # Combinaison des tags
            })

    # Convertir en DataFrame et sauvegarder en JSON
    dataframe = pd.DataFrame(annotations)
    dataframe.to_json(OUTPUT_JSON, orient="records", indent=4)
    print(f"Annotations enregistrées dans {OUTPUT_JSON}")

# Exécution de l'annotation
annotate_images()


Analyse de MAZDA%20AZ-1.JPG...
Analyse de 1991%20Hyundai%20Scoupe%201.5i%20GTX%20Automatic%20002.jpg...
Analyse de 1975%20Ford%20Landau%20%28P5%29%20coupe%20%2819102673449%29.jpg...
Analyse de SkodaSuperB.jpg...
Analyse de MHV%20Adler%20Standard%206S%201928%2001.jpg...
Analyse de Skoda%20440%20red%20vr%20TCE.jpg...
Analyse de .00%201505%201968%20Chevrolet%20Impala.jpg...
Analyse de 2nd%20Daihatsu%20Boon.jpg...
Analyse de 1980%20Mark%20VI%20Signature%20Series.jpg...
Analyse de Seat%20Arosa%20front%2020080722.jpg...
Analyse de 1987-1989%20Nissan%20Stanza%20GXE.jpg...
Analyse de Ford%20Sierra%20Bj.%20ca%201985%20photo%20Schaffen%20Diest%202017.jpg...
Analyse de Skoda%20Octavia%20I%20front%2020080213.jpg...
Analyse de 1955%20Skoda%201200%20%282568618690%29.jpg...
Analyse de 1971%20AMC%20Javelin%20SST%20red%20Kenosha%20street.JPG...
Analyse de Laurin%20%26%20Klement%20%E2%80%93%20%C5%A0koda%20350%20or%20360%20-%20cabriolet%20model.png...
Analyse de 1971%20Mercury%20Montego%20Cyclone.jpg...
